In [1]:
# ================================================
# 1. REPRODUCIBILITY SETTINGS 
# ================================================
from pathlib import Path

import random
import numpy as np
import pandas as pd
import torch

from tqdm import tqdm
import copy

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

# ----- Reproducibility -----
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ----- Device -----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ================================================
# 2. DEFAULT PATHS FOR DATASET
# ================================================
ROOT = Path("Amazon_products")   # dataset root directory

# Main corpus
TRAIN_CORPUS_PATH = ROOT / "train" /  "train_corpus.txt"       # pid \t text
TEST_CORPUS_PATH  = ROOT / "test" / "test_corpus.txt"        # pid \t text

# Taxonomy & class meta
CLASSES_PATH      = ROOT / "classes.txt"            # class_id \t class_name
HIERARCHY_PATH    = ROOT / "class_hierarchy.txt"    # parent_id \t child_id
KEYWORDS_PATH     = ROOT / "class_related_keywords.txt"

# Constants
NUM_CLASSES = 531
MIN_LABELS = 2     # minimum number of labels per sample
MAX_LABELS = 3     # maximum number of labels per sample

# Check paths
print("\n== Data path check ==")
for p in [TRAIN_CORPUS_PATH, TEST_CORPUS_PATH,
          CLASSES_PATH, HIERARCHY_PATH, KEYWORDS_PATH]:
    print(f"{p} -> {p.exists()}")

Device: cuda

== Data path check ==
Amazon_products/train/train_corpus.txt -> True
Amazon_products/test/test_corpus.txt -> True
Amazon_products/classes.txt -> True
Amazon_products/class_hierarchy.txt -> True
Amazon_products/class_related_keywords.txt -> True


In [2]:
# ================================================
# 2. DATA LOADING
# ================================================

def load_corpus(path):
    """
    Load corpus file (pid \\t text) as {pid: text} dictionary.
    """
    pid2text = {}
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split("\t", 1)
            if len(parts) == 2:
                pid, text = parts
                pid2text[pid] = text
    return pid2text

# Load corpus
print("Loading train/test corpus...")

pid2text_train = load_corpus(TRAIN_CORPUS_PATH)
pid2text_test  = load_corpus(TEST_CORPUS_PATH)

pid_list_train = list(pid2text_train.keys())
pid_list_test  = list(pid2text_test.keys())

print("Train samples:", len(pid2text_train))
print("Test samples :", len(pid2text_test))

# Quick sample check
for i, (pid, text) in enumerate(pid2text_train.items()):
    print(f"Example train sample #{i}: pid={pid}, text={text[:80]}...")
    break

Loading train/test corpus...
Train samples: 29487
Test samples : 19658
Example train sample #0: pid=0, text=omron hem 790it automatic blood pressure monitor with advanced omron health mana...


In [3]:
# ================================================
# 3. CLASS METADATA LOADING
# ================================================

def load_classes(path):
    """
    classes.txt : class_id \\t class_name
    returns: id2label, label2id
    """
    id2label = {}
    label2id = {}
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split("\t")
            if len(parts) != 2:
                continue
            cid, name = parts
            cid = int(cid)
            id2label[cid] = name
            label2id[name] = cid
    return id2label, label2id


def load_hierarchy(path):
    """
    class_hierarchy.txt : parent_id \\t child_id
    returns: edges (list of tuples)
    """
    edges = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split("\t")
            if len(parts) != 2:
                continue
            p, c = map(int, parts)
            edges.append((p, c))
    return edges


def load_keywords(path, label2id):
    """
    class_related_keywords.txt : CLASS_NAME: kw1, kw2,...
    returns: {class_id: [kws]}
    """
    d = {cid: [] for cid in label2id.values()}
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            if ":" not in line:
                continue
            name, kws = line.strip().split(":", 1)
            kws = [k.strip() for k in kws.split(",") if k.strip()]
            if name in label2id:
                cid = label2id[name]
                d[cid] = kws
    return d


# ----------------- Load all class meta -----------------
print("Loading class metadata...")

id2label, label2id = load_classes(CLASSES_PATH)
edges = load_hierarchy(HIERARCHY_PATH)
label_keywords = load_keywords(KEYWORDS_PATH, label2id)

print("Num classes:", len(id2label))
print("Num edges in taxonomy:", len(edges))
print()

# Small check
example_id = 0
print("Example class id:", example_id)
print("Name:", id2label[example_id])
print("Keywords:", label_keywords[example_id])

Loading class metadata...
Num classes: 531
Num edges in taxonomy: 568

Example class id: 0
Name: grocery_gourmet_food
Keywords: ['snacks', 'condiments', 'beverages', 'specialty_foods', 'spices', 'cooking_oils', 'baking_ingredients', 'gourmet_chocolates', 'artisanal_cheeses', 'organic_foods']


In [4]:
# ================================================
# 4. BERT EMBEDDING + DOC–CLASS SIMILARITY
# ================================================
import torch
import torch.nn.functional as F
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel

# ------------------------------------------------
# (1) BERT 모델 준비
# ------------------------------------------------
MODEL_NAME = "bert-base-uncased"  # 필요하면 다른 모델로 교체 가능 (ex. roberta-base 등)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
bert_model = AutoModel.from_pretrained(MODEL_NAME).to(device)
bert_model.eval()

print("Loaded BERT model:", MODEL_NAME, "| device:", device)

# ------------------------------------------------
# (2) mean pooling 함수 정의
# ------------------------------------------------
def mean_pooling(last_hidden_state, attention_mask):
    """
    last_hidden_state: [B, L, H]
    attention_mask   : [B, L]
    """
    mask = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
    sum_embeddings = (last_hidden_state * mask).sum(dim=1)          # [B, H]
    sum_mask = mask.sum(dim=1).clamp(min=1e-9)                      # [B, H]
    return sum_embeddings / sum_mask                                # [B, H]

@torch.no_grad()
def encode_texts_with_bert(texts, batch_size=32, max_length=128, desc="Encoding"):
    """
    texts: list of str
    return: torch.FloatTensor [N, H] (CPU 텐서)
    """
    all_embs = []
    for i in tqdm(range(0, len(texts), batch_size), desc=desc):
        batch = texts[i:i+batch_size]
        enc = tokenizer(
            batch,
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        ).to(device)

        outputs = bert_model(**enc)
        emb = mean_pooling(outputs.last_hidden_state, enc["attention_mask"])  # [B, H]
        all_embs.append(emb.cpu())

    return torch.cat(all_embs, dim=0)  # [N, H]


# ------------------------------------------------
# (3) train/test 문서 임베딩
# ------------------------------------------------
train_texts = [pid2text_train[pid] for pid in pid_list_train]
test_texts  = [pid2text_test[pid]  for pid in pid_list_test]

print("Encoding train documents with BERT...")
doc_emb_train = encode_texts_with_bert(
    train_texts,
    batch_size=32,
    max_length=128,
    desc="BERT encoding train docs"
)   # [N_train, H]

print("Encoding test documents with BERT...")
doc_emb_test = encode_texts_with_bert(
    test_texts,
    batch_size=32,
    max_length=128,
    desc="BERT encoding test docs"
)    # [N_test, H]

print("doc_emb_train:", doc_emb_train.shape)
print("doc_emb_test :", doc_emb_test.shape)


# ------------------------------------------------
# (4) 클래스 텍스트 (이름 + 키워드) 임베딩
# ------------------------------------------------
def build_class_text(cid):
    """
    클래스 이름 + 키워드를 한 문장으로 묶어서 BERT에 넣을 텍스트 생성
    """
    name = id2label[cid].replace("_", " ")
    kws  = " ".join(label_keywords.get(cid, []))
    if kws:
        return f"{name}. Related keywords: {kws}"
    else:
        return name

class_texts = [build_class_text(cid) for cid in range(NUM_CLASSES)]

print("Example class text 0:", class_texts[0])

print("Encoding class texts with BERT...")
class_emb = encode_texts_with_bert(
    class_texts,
    batch_size=32,
    max_length=32,
    desc="BERT encoding class texts"
)   # [C, H]

print("class_emb:", class_emb.shape)


# ------------------------------------------------
# (5) 코사인 유사도 기반 doc–class similarity 계산
# ------------------------------------------------
print("\nComputing BERT-based doc–class cosine similarities...")

# L2 정규화
doc_emb_train_norm = F.normalize(doc_emb_train, p=2, dim=1)   # [N_train, H]
class_emb_norm     = F.normalize(class_emb,     p=2, dim=1)   # [C, H]

# sims_train: [N_train, C]
sims = (doc_emb_train_norm @ class_emb_norm.T).numpy().astype("float32")

print("sims shape:", sims.shape)
print("sims min/max:", sims.min(), sims.max())
print("Done (BERT sims for train).")

/opt/conda/envs/esci/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded BERT model: bert-base-uncased | device: cuda
Encoding train documents with BERT...


BERT encoding train docs: 100%|██████████| 922/922 [01:27<00:00, 10.49it/s]


Encoding test documents with BERT...


BERT encoding test docs: 100%|██████████| 615/615 [01:00<00:00, 10.10it/s]


doc_emb_train: torch.Size([29487, 768])
doc_emb_test : torch.Size([19658, 768])
Example class text 0: grocery gourmet food. Related keywords: snacks condiments beverages specialty_foods spices cooking_oils baking_ingredients gourmet_chocolates artisanal_cheeses organic_foods
Encoding class texts with BERT...


BERT encoding class texts: 100%|██████████| 17/17 [00:00<00:00, 39.63it/s]


class_emb: torch.Size([531, 768])

Computing BERT-based doc–class cosine similarities...
sims shape: (29487, 531)
sims min/max: 0.23111407 0.8482967
Done (BERT sims for train).


In [6]:
# ================================================
# SAVE BERT EMBEDDINGS TO JSONL (one JSON per line)
# ================================================
import json
import numpy as np

def save_embeddings_jsonl(pid_list, emb_tensor, save_path):
    """
    pid_list   : list of IDs (ex. product IDs, class IDs)
    emb_tensor : torch.Tensor [N, H] (CPU or GPU)
    save_path  : output JSONL path
    """
    # numpy 변환 (GPU -> CPU 자동 전환)
    emb_np = emb_tensor.detach().cpu().numpy()

    with open(save_path, "w", encoding="utf-8") as f:
        for pid, vec in zip(pid_list, emb_np):
            item = {
                "id": pid,
                "embed": vec.tolist()      # JSON 직렬화를 위해 list로 변환
            }
            f.write(json.dumps(item) + "\n")

    print(f"[Saved] {save_path}  ({len(pid_list)} lines)")


# ------------------------------------------------
# 실제 저장 실행
# ------------------------------------------------

# 1) Train 문서 임베딩
save_embeddings_jsonl(
    pid_list_train,
    doc_emb_train,
    "train_doc_emb.jsonl"
)

# 2) Test 문서 임베딩
save_embeddings_jsonl(
    pid_list_test,
    doc_emb_test,
    "test_doc_emb.jsonl"
)

# 3) Class 임베딩
save_embeddings_jsonl(
    list(range(NUM_CLASSES)),   # class IDs = 0..530
    class_emb,
    "class_emb.jsonl"
)

[Saved] train_doc_emb.jsonl  (29487 lines)
[Saved] test_doc_emb.jsonl  (19658 lines)
[Saved] class_emb.jsonl  (531 lines)


In [7]:
# ================================================
# 5. SIMPLE TOP-K CANDIDATE SELECTION  (BERT version)
# ================================================

import numpy as np

def select_topk_candidates(sims_row, min_k=2, max_k=3):
    """
    sims_row: shape (C,) - one document's BERT-based cosine similarities
               ex) sims[i]  or sims_train[i]
    return: set of candidate class IDs
    """

    # sims_row 값이 모두 0 또는 음수일 수도 있으므로 np.nan 제거
    sims_clean = np.nan_to_num(sims_row, nan=0.0)

    # 1) similarity 기준 상위 max_k 선택
    top_k_idx = sims_clean.argsort()[-max_k:][::-1]   # 내림차순 정렬

    # 2) min_k 보장 (BERT 유사도가 낮은 문서에서도 최소 후보 보장)
    if len(top_k_idx) < min_k:
        top_k_idx = sims_clean.argsort()[-min_k:][::-1]

    # set 형태로 반환
    return set(map(int, top_k_idx))


# ------------------------------------------------
# Test (example)
# ------------------------------------------------
print("Simple Top-K candidate selection (BERT version) loaded.")

# 예시 출력: train 문서 0의 top-k 후보
print("Example (first document):", select_topk_candidates(sims[0], min_k=2, max_k=3))

Simple Top-K candidate selection (BERT version) loaded.
Example (first document): {145, 514, 241}


In [9]:
# ================================================
# 6. SIMPLE SILVER LABEL GENERATION (Top-k 방식, BERT sims)
# ================================================
import numpy as np
from tqdm import tqdm

def build_silver_labels_topk(sims, min_k=2, max_k=3):
    """
    sims : (N_train, NUM_CLASSES) BERT-based cosine similarity matrix
           ex) sims[i, j] = cos(doc_i, class_j)
    return:
        silver_y : (N, C) float32 (0/1 labels, Top-k)
    """
    N, C = sims.shape
    silver_y = np.zeros((N, C), dtype=np.float32)

    for i in tqdm(range(N), desc="Building silver labels (Top-k, BERT)"):
        # --- A. 안전하게 row 복사 ---
        row = np.array(sims[i], dtype=np.float32).copy()   # shape: (C,)

        # --- B. shape 검사 ---
        assert row.shape[0] == C, \
            f"Error: sims row shape {row.shape}, expected ({C},). Check sims axis order."

        # --- C. row 값이 전부 0인지 체크 (이상치 대비용) ---
        # BERT 코사인 유사도는 일반적으로 -1~1 사이 값이라
        # max==0 인 경우는 거의 없지만, NaN/전처리 문제 대비용
        if np.nanmax(row) == 0:
            # similarity 정보가 전혀 없다면 그냥 마지막 max_k개 인덱스를 fallback으로 사용
            topk_idx = np.arange(C)[-max_k:]
        else:
            # NaN을 0으로 치환
            row = np.nan_to_num(row, nan=0.0)

            # --- D. 정상 top-k 선택 ---
            # argsort ascending → 상위 max_k → 내림차순
            topk_idx = row.argsort()[-max_k:][::-1]

        # --- E. min_k 보장 ---
        if len(topk_idx) < min_k:
            topk_idx = row.argsort()[-min_k:][::-1]

        # --- F. silver label 기록 ---
        silver_y[i, topk_idx] = 1.0

    return silver_y


print("Generating silver labels using simple Top-k scheme with BERT sims...")

silver_labels_v1 = build_silver_labels_topk(
    sims,        # ✅ BERT 기반 similarity 행렬 사용
    min_k=2,
    max_k=3
)

print("silver_labels_v1 shape:", silver_labels_v1.shape)
print("avg positives per doc:", silver_labels_v1.sum(axis=1).mean())
print("Done.")

Generating silver labels using simple Top-k scheme with BERT sims...


Building silver labels (Top-k, BERT): 100%|██████████| 29487/29487 [00:01<00:00, 18766.84it/s]

silver_labels_v1 shape: (29487, 531)
avg positives per doc: 3.0
Done.


In [10]:
# ================================================
# 7. LABEL-GCN + DOCUMENT-CLASS CLASSIFIER (BERT 기반)
# ================================================
import torch
import torch.nn as nn
import torch.nn.functional as F

# ------------------------------------------------
# (1) Build normalized adjacency A_hat for GCN
# ------------------------------------------------
def build_normalized_adj(num_classes, edges):
    """
    edges: [(parent, child), ...]
    출력: A_hat (torch.FloatTensor, [C, C])
    """
    import numpy as np

    A = np.zeros((num_classes, num_classes), dtype=np.float32)

    # parent-child 연결을 양방향으로 넣기
    for p, c in edges:
        A[p, c] = 1.0
        A[c, p] = 1.0

    # self-loop
    np.fill_diagonal(A, 1.0)

    # D^{-1/2} * A * D^{-1/2}
    deg = A.sum(axis=1)
    deg_inv_sqrt = np.power(deg, -0.5)
    deg_inv_sqrt[np.isinf(deg_inv_sqrt)] = 0.0
    D_inv_sqrt = np.diag(deg_inv_sqrt)

    A_hat = D_inv_sqrt @ A @ D_inv_sqrt
    return torch.from_numpy(A_hat).float()


A_hat = build_normalized_adj(NUM_CLASSES, edges).to(device)
print("A_hat built:", A_hat.shape)


# ------------------------------------------------
# (2) Label Encoder: GCN (BERT 임베딩 입력)
# ------------------------------------------------
class LabelGCN(nn.Module):
    def __init__(self, in_dim, hidden_dim=256, num_layers=2, dropout=0.5):
        super().__init__()
        self.num_layers = num_layers
        self.dropout = nn.Dropout(dropout)
        
        dims = [in_dim] + [hidden_dim] * num_layers
        self.linears = nn.ModuleList()
        for i in range(num_layers):
            self.linears.append(nn.Linear(dims[i], dims[i+1]))

    def forward(self, A_hat, H):
        """
        H: [C, in_dim]  (클래스 초기 임베딩, 여기서는 BERT class_emb)
        """
        x = H  # [C, in_dim]
        for i, lin in enumerate(self.linears):
            x = A_hat @ x          # GCN aggregation
            x = lin(x)
            if i < self.num_layers - 1:
                x = F.relu(x)
                x = self.dropout(x)
        return x  # [C, hidden_dim]


# ------------------------------------------------
# (3) 전체 classifier: BERT doc → projection → dot with label GCN
# ------------------------------------------------
class TaxonomyClassifier(nn.Module):
    def __init__(self, emb_dim, hidden_dim=256):
        """
        emb_dim   : BERT 임베딩 차원 (예: 768)
        hidden_dim: GCN / 문서 임베딩 공통 latent 차원
        """
        super().__init__()
        # 문서 임베딩 projection: BERT_dim → hidden_dim
        self.doc_proj = nn.Linear(emb_dim, hidden_dim, bias=False)
        
        # 라벨 GCN: 초기 피처도 BERT_dim → hidden_dim
        self.label_gcn = LabelGCN(
            in_dim=emb_dim,     # label initial features = BERT class_emb
            hidden_dim=hidden_dim,
            num_layers=2,
            dropout=0.5
        )

    def forward(self, doc_feats, label_feats, A_hat):
        """
        doc_feats   : [N, emb_dim]   BERT 문서 임베딩 (doc_emb_train/test)
        label_feats : [C, emb_dim]   BERT 클래스 임베딩 (class_emb)
        A_hat       : [C, C]         taxonomy normalized adjacency
        """
        # 1) Document embedding
        doc_emb = self.doc_proj(doc_feats)             # [N, d]

        # 2) Label embedding via GCN
        label_emb = self.label_gcn(A_hat, label_feats) # [C, d]

        # 3) Matching score (bilinear의 단순 버전 = dot product)
        logits = doc_emb @ label_emb.T                 # [N, C]

        return logits, doc_emb, label_emb


print("BERT-based GNN classifier definitions loaded.")

A_hat built: torch.Size([531, 531])
BERT-based GNN classifier definitions loaded.


In [11]:
# ================================================
# 8. ROUND 1 TRAINING WITH SILVER LABELS v1 (BERT version)
# ================================================
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm

# ------------------------------------------------
# (1) Dataset: BERT doc_emb + silver labels
# ------------------------------------------------
class SilverDataset(Dataset):
    def __init__(self, doc_emb_tensor, y_np, pid_list=None):
        """
        doc_emb_tensor : torch.FloatTensor [N, H]  BERT 문서 임베딩
        y_np           : numpy array [N, C]        silver_labels_v1
        pid_list       : 옵션
        """
        self.X = doc_emb_tensor        # already dense
        self.y = torch.from_numpy(y_np).float()
        self.pid_list = pid_list

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


# ------------------------------------------------
# (2) Dataset / DataLoader
# ------------------------------------------------
train_dataset = SilverDataset(
    doc_emb_train,        # [N_train, emb_dim]
    silver_labels_v1,     # [N_train, 531]
    pid_list_train
)

batch_size = 64
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=True
)

print("Train dataset size:", len(train_dataset))
print("Batch size:", batch_size)


# ------------------------------------------------
# (3) 모델 / optimizer / loss 정의
# ------------------------------------------------
emb_dim = doc_emb_train.shape[1]   # 보통 768
hidden_dim = 256

model = TaxonomyClassifier(
    emb_dim=emb_dim,
    hidden_dim=hidden_dim
).to(device)

# Label GCN initial features = BERT class_emb
label_feats = class_emb.to(device)   # [C, emb_dim]

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


# ------------------------------------------------
# (4) Training loop  (Round 1)
# ------------------------------------------------
num_epochs = 2
logging_interval = 100   # 매 100스텝마다 loss 출력

for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0.0
    step = 0

    print(f"\n===== ROUND 1 — EPOCH {epoch} =====")
    for batch_x, batch_y in tqdm(train_loader, desc=f"Epoch {epoch}"):

        batch_x = batch_x.to(device)    # [B, emb_dim]
        batch_y = batch_y.to(device)    # [B, 531]

        optimizer.zero_grad()

        logits, doc_emb, label_emb = model(batch_x, label_feats, A_hat)

        loss = criterion(logits, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * batch_x.size(0)
        step += 1

        # 🔥 스텝별 loss 모니터링
        if step % logging_interval == 0:
            print(f"  Step {step} — loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_dataset)
    print(f"[Epoch {epoch}] AVG LOSS = {avg_loss:.6f}")

print("\nRound 1 training finished.")

Train dataset size: 29487
Batch size: 64

===== ROUND 1 — EPOCH 1 =====


Epoch 1:  49%|████▊     | 224/461 [00:00<00:00, 600.39it/s]

  Step 100 — loss: 0.0307
  Step 200 — loss: 0.0290


Epoch 1:  92%|█████████▏| 422/461 [00:00<00:00, 640.00it/s]

  Step 300 — loss: 0.0287
  Step 400 — loss: 0.0248


...

  Step 300 — loss: 0.0224


Epoch 2:  85%|████████▌ | 394/461 [00:00<00:00, 645.85it/s]

  Step 400 — loss: 0.0220


Epoch 2: 100%|██████████| 461/461 [00:00<00:00, 639.93it/s]

[Epoch 2] AVG LOSS = 0.022487

Round 1 training finished.


In [13]:
# ================================================
# 9. SELF-TRAINING: BUILD SILVER LABELS v2 FROM ROUND 1 MODEL (BERT)
# ================================================
import numpy as np
from tqdm import tqdm

# ------------------------------------------------
# (1) Round 1 prediction 함수 (train 전체 predict, BERT 버전)
# ------------------------------------------------
def predict_train_probs(model, doc_emb_tensor, label_feats, A_hat, batch_size=64):
    """
    Train 전체 문서에 대해 확률 예측을 반환한다.
    doc_emb_tensor : torch.FloatTensor [N_train, emb_dim] (doc_emb_train)
    label_feats    : torch.FloatTensor [C, emb_dim] (class_emb)
    A_hat          : [C, C] normalized adjacency
    return:
        probs : numpy array (N_train, NUM_CLASSES)
    """
    model.eval()
    N = doc_emb_tensor.size(0)
    all_probs = []

    with torch.no_grad():
        for i in tqdm(range(0, N, batch_size), desc="Predicting Round 1"):
            batch_x = doc_emb_tensor[i : i+batch_size].to(device)  # [B, emb_dim]

            logits, _, _ = model(batch_x, label_feats, A_hat)      # [B, C]
            probs = torch.sigmoid(logits).cpu().numpy()

            all_probs.append(probs)

    return np.vstack(all_probs)


# ------------------------------------------------
# (2) Round 1 전체 예측 수행
# ------------------------------------------------
print("Predicting train probabilities (Round 1)...")

probs_train = predict_train_probs(
    model,
    doc_emb_train,   # ✅ BERT 문서 임베딩 사용
    label_feats,     # class_emb.to(device)
    A_hat,
    batch_size=64
)

print("probs_train shape:", probs_train.shape)  # (N_train, 531)
print("probs range:", probs_train.min(), probs_train.max())


# ------------------------------------------------
# (3) Silver labels v2 생성
#     HIGH = 0.7  → confident positive
#     LOW  = 0.3  → confident negative
# ------------------------------------------------
def build_silver_labels_v2(probs, silver_v1, high=0.7, low=0.3):
    """
    probs     : round 1 predicted probabilities (N, C)
    silver_v1 : previous silver labels (N, C)
    return:
        silver_v2 (N, C)
    """
    silver_v2 = silver_v1.copy()

    pos_mask = probs >= high    # 모델이 강하게 1이라 보는 위치
    neg_mask = probs <= low     # 모델이 강하게 0이라 보는 위치

    silver_v2[pos_mask] = 1.0
    silver_v2[neg_mask] = 0.0

    return silver_v2


silver_labels_v2 = build_silver_labels_v2(
    probs_train,
    silver_labels_v1,
    high=0.7,
    low=0.3
)

print("silver_labels_v2 shape:", silver_labels_v2.shape)
print("avg positives per doc (v1):", silver_labels_v1.sum(axis=1).mean())
print("avg positives per doc (v2):", silver_labels_v2.sum(axis=1).mean())
print("Self-training Round 2 labels ready.")

Predicting train probabilities (Round 1)...


Predicting Round 1: 100%|██████████| 461/461 [00:00<00:00, 2784.24it/s]

probs_train shape: (29487, 531)
probs range: 1.481398e-15 0.99778086


silver_labels_v2 shape: (29487, 531)
avg positives per doc (v1): 3.0
avg positives per doc (v2): 0.6142707
Self-training Round 2 labels ready.


In [14]:
# ================================================
# 10. ROUND 2 TRAINING WITH SILVER LABELS v2 (BERT version)
# ================================================
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from tqdm import tqdm

# SilverDataset(BERT용)는 8번 셀에서 이미 정의되어 있음:
# class SilverDataset(Dataset):
#     def __init__(self, doc_emb_tensor, y_np, pid_list=None):
#         ...
#     def __len__(self):
#         ...
#     def __getitem__(self, idx):
#         ...

# ------------------------------------------------
# (1) Round 2용 Dataset / DataLoader 생성
# ------------------------------------------------
train_dataset_v2 = SilverDataset(
    doc_emb_train,    # ✅ BERT 문서 임베딩
    silver_labels_v2, # ✅ Round 2 silver labels
    pid_list_train
)

batch_size_round2 = 64
train_loader_v2 = DataLoader(
    train_dataset_v2,
    batch_size=batch_size_round2,
    shuffle=True,
    num_workers=0,
    pin_memory=True
)

print("Round 2 train dataset size:", len(train_dataset_v2))
print("Round 2 batch size:", batch_size_round2)


# ------------------------------------------------
# (2) Optimizer / Loss 재설정 (모델은 Round 1에서 이어서 사용)
# ------------------------------------------------
criterion_round2 = nn.BCEWithLogitsLoss()
optimizer_round2 = torch.optim.Adam(model.parameters(), lr=1e-3)


# ------------------------------------------------
# (3) Round 2 Training Loop
# ------------------------------------------------
num_epochs_round2 = 2          # Round 2에서도 1~2 epoch 정도 추천
logging_interval = 100         # step마다 loss 모니터링 간격

for epoch in range(1, num_epochs_round2 + 1):
    model.train()
    total_loss = 0.0
    step = 0

    print(f"\n===== ROUND 2 — EPOCH {epoch} =====")
    for batch_x, batch_y in tqdm(train_loader_v2, desc=f"[Round 2] Epoch {epoch}"):
        batch_x = batch_x.to(device)   # [B, emb_dim]
        batch_y = batch_y.to(device)   # [B, C]

        optimizer_round2.zero_grad()

        # label_feats, A_hat, model 은 Round 1에서 이미 정의된 것을 사용
        logits, doc_emb, label_emb = model(batch_x, label_feats, A_hat)  # [B, C]
        loss = criterion_round2(logits, batch_y)

        loss.backward()
        optimizer_round2.step()

        total_loss += loss.item() * batch_x.size(0)
        step += 1

        # 🔍 중간 loss 모니터링
        if step % logging_interval == 0:
            print(f"  [Round 2] Step {step} — loss: {loss.item():.6f}")

    avg_loss = total_loss / len(train_dataset_v2)
    print(f"[Round 2 - Epoch {epoch}] AVG LOSS = {avg_loss:.6f}")

print("\nRound 2 training finished.")

Round 2 train dataset size: 29487
Round 2 batch size: 64

===== ROUND 2 — EPOCH 1 =====


[Round 2] Epoch 1:  13%|█▎        | 59/461 [00:00<00:00, 582.67it/s]

  [Round 2] Step 100 — loss: 0.001564


[Round 2] Epoch 1:  39%|███▉      | 180/461 [00:00<00:00, 596.05it/s]

  [Round 2] Step 200 — loss: 0.001911


...

  [Round 2] Step 300 — loss: 0.001239


[Round 2] Epoch 2:  84%|████████▍ | 389/461 [00:00<00:00, 635.42it/s]

  [Round 2] Step 400 — loss: 0.001220


[Round 2] Epoch 2: 100%|██████████| 461/461 [00:00<00:00, 635.48it/s]

[Round 2 - Epoch 2] AVG LOSS = 0.001863

Round 2 training finished.


In [16]:
# ================================================
# 11. FINAL TEST PREDICTION + SUBMISSION (BERT + GCN)
# ================================================
import csv
import numpy as np
from tqdm import tqdm
import torch
import torch.nn.functional as F

SUBMISSION_PATH = "submission.csv"  # output file

# ------------------------------------------------
# (1) Test 전체에 대한 확률 예측 (Round 2 model)
# ------------------------------------------------
def predict_test_probs(model, doc_emb_tensor, label_feats, A_hat, batch_size=64):
    """
    doc_emb_tensor : torch.FloatTensor [N_test, emb_dim] (doc_emb_test)
    label_feats    : torch.FloatTensor [C, emb_dim] (class_emb)
    A_hat          : [C, C]
    return:
        probs_test : numpy array [N_test, NUM_CLASSES]
    """
    model.eval()
    N = doc_emb_tensor.size(0)
    all_probs = []

    with torch.no_grad():
        for i in tqdm(range(0, N, batch_size), desc="Predicting on test"):
            batch_x = doc_emb_tensor[i : i+batch_size].to(device)  # [B, emb_dim]

            logits, _, _ = model(batch_x, label_feats, A_hat)      # [B, C]
            probs = torch.sigmoid(logits).cpu().numpy()
            all_probs.append(probs)

    return np.vstack(all_probs)


print("Predicting test probabilities with Round 2 model...")

probs_test = predict_test_probs(
    model,
    doc_emb_test,   # ✅ BERT test 문서 임베딩
    label_feats,    # ✅ class_emb.to(device)
    A_hat,
    batch_size=64
)

print("probs_test shape:", probs_test.shape)
print("probs_test range:", probs_test.min(), probs_test.max())


# ------------------------------------------------
# (2) BERT cosine similarity 기반 sims_test 계산
# ------------------------------------------------
print("\nComputing BERT-based doc–class sims for test set...")

# doc_emb_test, class_emb 둘 다 같은 공간(BERT)에서 normalize
doc_emb_test_norm = F.normalize(doc_emb_test, p=2, dim=1)        # [N_test, H]
class_emb_norm    = F.normalize(class_emb,    p=2, dim=1)        # [C, H]

# torch matmul → numpy
sims_test = torch.matmul(
    doc_emb_test_norm.to(device),
    class_emb_norm.to(device).T
).cpu().numpy().astype("float32")                                # [N_test, C]

print("sims_test shape:", sims_test.shape)
print("sims_test range:", sims_test.min(), sims_test.max())


# ------------------------------------------------
# (3) MODEL + SIMS BLENDING
# ------------------------------------------------
lambda_model = 0.6
lambda_sims  = 0.4

final_scores = lambda_model * probs_test + lambda_sims * sims_test
print("\nfinal_scores shape:", final_scores.shape)
print("final_scores range:", final_scores.min(), final_scores.max())


# ------------------------------------------------
# (4) 동적 라벨 선택 함수: 문서당 1~3개 (threshold 기반)
# ------------------------------------------------
def pick_labels(score_row, min_k=1, max_k=3, threshold=0.5):
    """
    score_row : shape (C,) - 한 문서의 최종 점수 벡터
    min_k     : 최소 라벨 개수 (보통 1)
    max_k     : 최대 라벨 개수 (보통 3)
    threshold : 이 값 이상만 '유력 후보'로 먼저 선택
    """
    # 1) threshold 이상인 클래스만 후보
    cand = np.where(score_row >= threshold)[0]

    # 2) threshold 넘는 게 하나도 없으면 → top-1 강제 선택
    if len(cand) == 0:
        top1 = int(np.argmax(score_row))
        return [top1]

    # 3) 후보가 너무 많으면 → 점수 순으로 max_k개만 남기기
    if len(cand) > max_k:
        sorted_idx = cand[np.argsort(score_row[cand])[::-1]]
        cand = sorted_idx[:max_k]

    # 4) 후보가 min_k보다 적으면 → top-score에서 채워서 min_k까지 맞추기
    if len(cand) < min_k:
        sorted_idx = np.argsort(score_row)[::-1]
        for c in sorted_idx:
            if c not in cand:
                cand = np.append(cand, c)
                if len(cand) == min_k:
                    break

    return list(map(int, cand))


# ------------------------------------------------
# (5) SUBMISSION 생성 (Kaggle baseline 포맷: id, labels)
# ------------------------------------------------
all_pids, all_labels = [], []

print("\nGenerating final predictions for submission.csv...")
for i, pid in enumerate(tqdm(pid_list_test, desc="Scoring test instances")):
    scores = final_scores[i]
    labels = pick_labels(
        scores,
        min_k=MIN_LABELS,   # 보통 1
        max_k=MAX_LABELS,   # 보통 3
        threshold=0.5       # 필요하면 조정 (0.4~0.6 사이 튜닝 가능)
    )
    labels = sorted(labels)
    all_pids.append(pid)
    all_labels.append(labels)

# 라벨 개수 분포 한번 체크 (선택사항, 디버깅용)
lengths = [len(l) for l in all_labels]
print("\nLabel count per doc:")
for L in sorted(set(lengths)):
    print(f"  {L} labels : {lengths.count(L)} docs")

# --- Save submission file ---
with open(SUBMISSION_PATH, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["id", "labels"])   # ✅ Kaggle이 요구하는 컬럼 이름

    for pid, labels in zip(all_pids, all_labels):
        writer.writerow([pid, ",".join(map(str, labels))])

print(f"\nSubmission file saved to: {SUBMISSION_PATH}")
print(f"Total samples: {len(all_pids)}, Classes per sample: {MIN_LABELS}-{MAX_LABELS} (dynamic)")

Predicting test probabilities with Round 2 model...


Predicting on test: 100%|██████████| 308/308 [00:00<00:00, 3363.96it/s]


probs_test shape: (19658, 531)
probs_test range: 2.6146454e-37 1.0

Computing BERT-based doc–class sims for test set...
sims_test shape: (19658, 531)
sims_test range: 0.25432742 0.8487386

final_scores shape: (19658, 531)
final_scores range: 0.101731725 0.92894423

Generating final predictions for submission.csv...


Scoring test instances: 100%|██████████| 19658/19658 [00:00<00:00, 40681.29it/s]


Label count per doc:
  1 labels : 9461 docs
  2 labels : 10197 docs

Submission file saved to: submission.csv
Total samples: 19658, Classes per sample: 2-3 (dynamic)


In [ ]:
# ------------------------
# Dummy baseline for Kaggle submission
# Generates random multi-label predictions
# ------------------------
import os
import csv
import random
from tqdm import tqdm

# --- Paths ---
TEST_DIR = "Amazon_products/test"  # modify if needed
TEST_CORPUS_PATH = os.path.join(TEST_DIR, "test_corpus.txt")  # product_id \t text
SUBMISSION_PATH = "submission.csv"  # output file

# --- Constants ---
NUM_CLASSES = 531  # total number of classes (0–530)
MIN_LABELS = 1     # minimum number of labels per sample
MAX_LABELS = 3     # maximum number of labels per sample

# --- Load test corpus ---
def load_corpus(path):
    """Load test corpus into {pid: text} dictionary."""
    pid2text = {}
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split("\t", 1)
            if len(parts) == 2:
                pid, text = parts
                pid2text[pid] = text
    return pid2text

pid2text_test = load_corpus(TEST_CORPUS_PATH)
pid_list_test = list(pid2text_test.keys())

# --- Generate random predictions ---
all_pids, all_labels = [], []
for pid in tqdm(pid_list_test, desc="Generating dummy predictions"):
    n_labels = random.randint(MIN_LABELS, MAX_LABELS)
    labels = random.sample(range(NUM_CLASSES), n_labels)
    labels = sorted(labels)
    all_pids.append(pid)
    all_labels.append(labels)

# --- Save submission file ---
with open(SUBMISSION_PATH, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["pid", "labels"])
    for pid, labels in zip(all_pids, all_labels):
        writer.writerow([pid, ",".join(map(str, labels))])

print(f"Dummy submission file saved to: {SUBMISSION_PATH}")
print(f"Total samples: {len(all_pids)}, Classes per sample: {MIN_LABELS}-{MAX_LABELS}")

Generating dummy predictions: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19658/19658 [00:00<00:00, 190266.11it/s]

Dummy submission file saved to: submission.csv
Total samples: 19658, Classes per sample: 1-3
